In [33]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager 

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://public.tableau.com/app/discover/authors")
print(driver.title)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\User\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [53]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
base = 'https://public.tableau.com'
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")


## Ref - https://stackoverflow.com/questions/63535607/how-to-scrape-the-data-from-multiple-same-class-under-div-using-beautifulsoup
## Above one saved me to get numbers from same class names 
div_class = soup.find_all('div', class_='FeaturedAuthorsList_featuredAuthorsGridItem__1Dgam')
all_dev = []

from create_db import create_connection,create_table
database = 'test.db'
conn = create_connection(database)

for i in div_class:
    try:
        name = i.find('div', class_='ProfileCard_namePlate__164ez').get_text()
        address = i.find('div', class_='ProfileCard_address__1Qg-J').get_text()
        total_viz = i.find('span', string='vizzes').parent.find(class_='ProfileCard_statCount__3V8ah').get_text()
        follower = i.find('span',string='followers').parent.find(class_='ProfileCard_statCount__3V8ah').get_text()
        following = i.find('span',string='following').parent.find(class_='ProfileCard_statCount__3V8ah').get_text()
        href = i.find('a', class_='ProfileCard_link__2n0JF').attrs['href']
        url = urljoin(base,href)
        # data = (name,address,total_viz,follower,following,url)
        # create_table(conn,data)

        feature_dev = {}
        feature_dev['Name'] = name 
        feature_dev['Address'] = address
        feature_dev['Total_vizzes'] = total_viz
        feature_dev['Followers'] = follower 
        feature_dev['Following'] = following 
        feature_dev['Profile_URL'] = url
        all_dev.append(feature_dev) 
    except:
        print("Error")
print(all_dev)



[{'Name': 'Idris Akilapa', 'Address': 'Ibadan, Nigeria', 'Total_vizzes': '51', 'Followers': '292', 'Following': '86', 'Profile_URL': 'https://public.tableau.com/app/profile/akilapa.idris5302'}, {'Name': 'Anita Onyeacho', 'Address': 'Charlotte, United States', 'Total_vizzes': '23', 'Followers': '213', 'Following': '181', 'Profile_URL': 'https://public.tableau.com/app/profile/anita.onyeacho'}, {'Name': 'Ant Pulley', 'Address': 'Letchworth Garden City, United Kingdom', 'Total_vizzes': '75', 'Followers': '318', 'Following': '351', 'Profile_URL': 'https://public.tableau.com/app/profile/ant.pulley'}, {'Name': 'Bill Chen (陳克勤)', 'Address': 'TAOYUAN, Taiwan', 'Total_vizzes': '99', 'Followers': '289', 'Following': '451', 'Profile_URL': 'https://public.tableau.com/app/profile/bill1210'}, {'Name': 'Dennis Kao', 'Address': 'Ottawa, Canada', 'Total_vizzes': '79', 'Followers': '131', 'Following': '78', 'Profile_URL': 'https://public.tableau.com/app/profile/dennis.kao'}, {'Name': 'Ervin Vinzon', 'Add

In [54]:
import pandas as pd
df = pd.DataFrame(all_dev)
print(df.head())




              Name                                 Address Total_vizzes  \
0    Idris Akilapa                         Ibadan, Nigeria           51   
1   Anita Onyeacho                Charlotte, United States           23   
2       Ant Pulley  Letchworth Garden City, United Kingdom           75   
3  Bill Chen (陳克勤)                         TAOYUAN, Taiwan           99   
4       Dennis Kao                          Ottawa, Canada           79   

  Followers Following                                        Profile_URL  
0       292        86  https://public.tableau.com/app/profile/akilapa...  
1       213       181  https://public.tableau.com/app/profile/anita.o...  
2       318       351  https://public.tableau.com/app/profile/ant.pulley  
3       289       451    https://public.tableau.com/app/profile/bill1210  
4       131        78  https://public.tableau.com/app/profile/dennis.kao  


In [59]:
## REF - https://towardsdatascience.com/sqlalchemy-python-tutorial-79a577141a91
## Above tutorial helped me to write below code , but I still need to try to understand how SQLalchemy works
import sqlalchemy as db
from sqlalchemy import create_engine

database = "test.db"
sqlite_url = f"sqlite:///{database}"
engine = create_engine(sqlite_url)

connection = engine.connect()
metadata = db.MetaData()
dev = db.Table('dev',metadata, autoload=True, autoload_with=engine)

In [63]:
query = db.select([dev])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
print(ResultSet)

In [66]:
## Need to change if_exists='append' if I run next time
df.to_sql('dev', con=engine, if_exists='replace')

18

In [67]:
engine.execute("SELECT * FROM dev").fetchall()

[(0, 'Idris Akilapa', 'Ibadan, Nigeria', '51', '292', '86', 'https://public.tableau.com/app/profile/akilapa.idris5302'),
 (1, 'Anita Onyeacho', 'Charlotte, United States', '23', '213', '181', 'https://public.tableau.com/app/profile/anita.onyeacho'),
 (2, 'Ant Pulley', 'Letchworth Garden City, United Kingdom', '75', '318', '351', 'https://public.tableau.com/app/profile/ant.pulley'),
 (3, 'Bill Chen (陳克勤)', 'TAOYUAN, Taiwan', '99', '289', '451', 'https://public.tableau.com/app/profile/bill1210'),
 (4, 'Dennis Kao', 'Ottawa, Canada', '79', '131', '78', 'https://public.tableau.com/app/profile/dennis.kao'),
 (5, 'Ervin Vinzon', 'Manila, Philippines', '85', '163', '150', 'https://public.tableau.com/app/profile/ervin.vinzon'),
 (6, 'Harim Jung', '', '87', '236', '157', 'https://public.tableau.com/app/profile/harim.jung'),
 (7, 'Jared Flores', 'Dallas, United States', '27', '185', '233', 'https://public.tableau.com/app/profile/jared8766'),
 (8, 'Juan Angel Martínez Arzola', 'Monterrey, Mexico'